In [ ]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al azar''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# NearestNeighbors se usará para mejorar la búsqueda de puntos cercanos.
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(metric='euclidean')

In [ ]:
class FlockAgent(Agent):
    def __init__(self, unique_id, model, x, y, width, height):
        super().__init__(unique_id, model)
        
        # Representa la posición del agente usando un vector 2D
        self.position = np.array((x, y), dtype=np.float64)
    
        # Generamos aleatoriamente un vector que representa la velocidad del agente.
        vec = (np.random.rand(2) - 0.5)*10
        self.velocity = np.array(vec, dtype=np.float64)
        
        # Generamos aleatoriamente un vector que representa la aceleración del agente.
        vec = (np.random.rand(2) - 0.5)/2
        self.acceleration = np.array(vec, dtype=np.float64)
        
        # Aceleración que deben de tener para poder alinearse.
        self.max_force = 0.3
        
        # Magnitud máxima que puede tener el vector de velocidad.
        self.max_speed = 5
        
        # Distancia percibida como adecuada por el agente.
        self.perception = 50
        
        self.width = width
        self.height = height
        
        self.agents_index = None
        
    def step(self):
        self.check_edges()
        
        self.agents_index = neighbors.radius_neighbors([self.position], self.perception)[1][0]
        
        self.check_with_neighbours()
        
        self.position = self.position + self.velocity
        self.velocity = self.velocity + self.acceleration
        
        # Limitaremos la velocidad máxima. Para ello, vamos a normalizar el vector para verificar que su magnitud
        # no exceda de un tope definido.
        if np.linalg.norm(self.velocity) > self.max_speed:
            self.velocity = self.velocity / np.linalg.norm(self.velocity) * self.max_speed
            
        self.acceleration = np.array([0, 0], dtype=np.float64)
        